In [1]:
import pandas as pd
import requests
from sqlalchemy import types, create_engine
from sqlalchemy.exc import InvalidRequestError
from bs4 import BeautifulSoup
connection_url = 'mysql+pymysql://root:DimaP/120897@localhost:3306/test'
engine = create_engine(connection_url)

try:
    engine.connect()
    print("Connection successful!")
except Exception as e:
    print("Connection failed:", str(e))

Connection successful!


In [5]:
url = "https://api.apilayer.com/exchangerates_data/latest?base=USD"
payload = {}
headers= {
  "apikey": "Z0P3aS85iISgHr1zKnDLs1GUkmTaIHm2"
}
response = requests.request("GET", url, headers=headers, data = payload)
result = pd.DataFrame(response.json()).reset_index()
result = result[['index','rates']]
result_types = dict(zip(
    result.columns.tolist(),
    (types.VARCHAR(length=3), types.Float()) ))

try:
    result.to_sql( name="currency", schema="test"
              , con=engine
              , if_exists='replace'
              , index=False
              , dtype = result_types)
except (ValueError, InvalidRequestError):
    print ("Could not write to table 'test'.")

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)#IMF_estimates_between_2010_and_2019'
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find_all('table')[4]
headers = ['country']+[i for i in range(2010,2020)]
data = []
for row in table.find_all('tr'):
    row_data = []
    for cell in row.find_all('td'):
        row_data.append(cell.text.strip())
    if row_data:
        row_data = [int(int(value.replace(',', ''))*1000000) if value.replace(',', '').isdigit() else value for value in row_data]
        data.append(row_data)
gdp = pd.DataFrame(data, columns=headers).sort_values(by=2019, ascending=False).head().reset_index(drop=True)
gdp = gdp.melt(id_vars='country', var_name='year', value_name='gdp')
gdp_types = dict(zip(
    gdp.columns.tolist(),
    (types.VARCHAR(length=100), types.INT(),types.BIGINT()) ))
try:
    gdp.to_sql( name="gdp", schema="test"
              , con=engine
              , if_exists='replace'
              , index=False
              , dtype = gdp_types)
except (ValueError, InvalidRequestError):
    print ("Could not write to table 'test'.")

In [8]:
info = {'United States': 'https://www.macrotrends.net/countries/USA/united-states/population', 
        'China':'https://www.macrotrends.net/countries/CHN/china/population', 
        'Japan':'https://www.macrotrends.net/countries/JPN/japan/population', 
        'Germany':'https://www.macrotrends.net/countries/DEU/germany/population',
        'United Kingdom':'https://www.macrotrends.net/countries/GBR/united-kingdom/population'}
popul = pd.DataFrame()
for name, url in info.items():
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find_all('table')[1]
    data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        if row_data:
            if int(row_data[0]) in [i for i in range(2010,2020)]:
                    data.append([int(row_data[0]), int(row_data[1].replace(',',''))])
    df_temp = pd.DataFrame(data, columns=['year', 'population']).assign(country=name)
    popul = pd.concat([popul,df_temp], axis=0, ignore_index=True)
popul_types = dict(zip(
    popul.columns.tolist(),
    (types.INT(), types.INT(),types.VARCHAR(length=100)) ))
try:
    popul.to_sql( name="popul", schema="test"
              , con=engine
              , if_exists='replace'
              , index=False
              , dtype = popul_types)
except (ValueError, InvalidRequestError):
    print ("Could not write to table 'test'.")